In [93]:
import pandas_datareader.data as web
from fredapi import Fred


import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import hstack
from scipy import stats
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV, ElasticNet, LassoLarsCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

import warnings
import os

warnings.filterwarnings('ignore')

import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
import pmdarima as pm
from pmdarima import model_selection

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

%matplotlib inline

In [266]:
start = datetime(2000,3,1)
end = datetime(2020,1,31)

In [267]:
AMT = web.DataReader('AMT', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
AMT = AMT.copy()
AMT.index = pd.to_datetime(AMT.index)
AMT = AMT.resample('MS').mean()
AMT['name'] = pd.Series()
AMT['name'] = 'AMT'


ELS = web.DataReader('ELS', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
ELS = ELS.copy()
ELS.index = pd.to_datetime(ELS.index)
ELS = ELS.resample('MS').mean()
ELS['name'] = pd.Series()
ELS['name'] = 'ELS'

PLD = web.DataReader('PLD', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
PLD = PLD.copy()
PLD.index = pd.to_datetime(PLD.index)
PLD = PLD.resample('MS').mean()
PLD['name'] = pd.Series()
PLD['name'] = 'PLD'

FR = web.DataReader('FR', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
FR = FR.copy()
FR.index = pd.to_datetime(FR.index)
FR = FR.resample('MS').mean()
FR['name'] = pd.Series()
FR['name'] = 'FR'


In [292]:
MAA = web.DataReader('MAA', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
MAA = MAA.copy()
MAA.index = pd.to_datetime(MAA.index)
MAA = MAA.resample('MS').mean()
MAA['name'] = pd.Series()
MAA['name'] = 'MAA'

SUI = web.DataReader('SUI', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
SUI = SUI.copy()
SUI.index = pd.to_datetime(SUI.index)
SUI = SUI.resample('MS').mean()
SUI['name'] = pd.Series()
SUI['name'] = 'SUI'

BXMT = web.DataReader('BXMT', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
BXMT = BXMT.copy()
BXMT.index = pd.to_datetime(BXMT.index)
BXMT = BXMT.resample('MS').mean()
BXMT['name'] = pd.Series()
BXMT['name'] = 'BXMT'

RHP = web.DataReader('RHP', 'av-daily', start = start, end = end, api_key = 'Y8YQFOIVHPA381U4')
RHP = RHP.copy()
RHP.index = pd.to_datetime(RHP.index)
RHP = RHP.resample('MS').mean()
RHP['name'] = pd.Series()
RHP['name'] = 'RHP'


In [293]:
reit_list = [AMT, PLD, ELS, FR, MAA, SUI, BXMT, RHP]
for reit in reit_list:
    reit['mid'] = (reit['high'] + reit['low'])/2
    reit.columns = ['open', 'high', 'low', 'close', 'volume', 'name', 'mid']
    reit.drop(['open', 'high', 'low', 'close', 'volume', 'name'], axis = 1, inplace = True)

## Exogenous Data Gathering

In [294]:
interest_rates = 0

In [295]:
fred = Fred(api_key = 'aec5f13aa40d1af4edd587ffccceeb1a')

In [296]:
# LIBOR rate (USD1MTD156N)  
usd1mtd156n = fred.get_series('usd1mtd156n', start, end)
usd1mtd156n = round(usd1mtd156n.resample('MS').mean(),2)
usd1mtd156n = pd.DataFrame(usd1mtd156n, index = usd1mtd156n.index, columns = ['usd1mtd156n'])
interest_rates += 1

In [297]:
# 30 Year Fixed Rate Mortgage (MORTGAGE30US)
mortgage30us = fred.get_series('MORTGAGE30US', start, end)
mortgage30us = round(mortgage30us.resample('MS').mean(),2)
mortgage30us = pd.DataFrame(mortgage30us, index = mortgage30us.index, columns = ['mortgage30us'])
interest_rates += 1

In [298]:
# 15 Year Fixed rate Mortgage (MORTGAGE15US)
mortgage15us = fred.get_series('MORTGAGE15US', start, end)
mortgage15us = round(mortgage15us.resample('MS').mean(),2)
mortgage15us = pd.DataFrame(mortgage15us, index = mortgage15us.index, columns = ['mortgage15us'])
interest_rates += 1

In [299]:
# 3-Month AA Financial Commercial Paper Rate (DCPF3M)
dcpf3m = fred.get_series('DCPF3M', start, end)
dcpf3m = round(dcpf3m.resample('MS').mean(),2)
dcpf3m = pd.DataFrame(dcpf3m, index = dcpf3m.index, columns = ['dcpf3m'])
interest_rates += 1

In [300]:
# 3-Month AA Nonfinancial Commercial Paper Rate (CPN3M)
cpn3m = fred.get_series('cpn3m', start, end)
cpn3m = round(cpn3m.resample('MS').mean(),2)
cpn3m = pd.DataFrame(cpn3m, index = dcpf3m.index, columns = ['cpn3m'])
interest_rates += 1

In [301]:
# 1-Month AA Financial Commercial Paper Rate (DCPF1M)
dcpf1m = fred.get_series('dcpf1m', start, end)
dcpf1m = round(dcpf1m.resample('MS').mean(),2)
dcpf1m = pd.DataFrame(dcpf1m, index = dcpf3m.index, columns = ['dcpf1m'])
interest_rates += 1

In [302]:
# Moody's Seasoned Aaa Corporate Bond Yield (AAA)
aaa = fred.get_series('AAA', start, end)
aaa = round(aaa.resample('MS').mean(),2)
aaa = pd.DataFrame(aaa, index = dcpf3m.index, columns = ['aaa'])
interest_rates += 1

In [303]:
# Moody's Seasoned Baa Corporate Bond Yield (BAA)
baa = fred.get_series('BAA', start, end)
baa = round(baa.resample('MS').mean(),2)
baa = pd.DataFrame(baa, index = baa.index, columns = ['baa'])
interest_rates += 1

In [304]:
# Effective Federal Funds Rate (DFF)
dff = fred.get_series('DFF', start, end)
dff = round(dff.resample('MS').mean(),2)
dff = pd.DataFrame(dff, index = dff.index, columns = ['dff'])
interest_rates += 1

In [305]:
# Eighth District Flexible Rate on Seasonal Credit (FLEXSC)
flexsc = fred.get_series('flexsc', start, end)
flexsc = round(flexsc.resample('MS').mean(),2)
flexsc = pd.DataFrame(flexsc, index = flexsc.index, columns = ['flexsc'])
interest_rates += 1

In [306]:
# 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity (T10Y2Y)
t10y2y = fred.get_series('t10y2y', start, end)
t10y2y = round(t10y2y.resample('MS').mean(),2)
t10y2y = pd.DataFrame(t10y2y, index = t10y2y.index, columns = ['t10y2y'])
interest_rates += 1

In [307]:
# 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity (T10Y3M)
t10y3m = fred.get_series('t10y3m', start, end)
t10y3m = round(t10y3m.resample('MS').mean(),2)
t10y3m = pd.DataFrame(t10y3m, index = t10y3m.index, columns = ['t10y3m'])
interest_rates += 1

In [308]:
# TED Spread (TEDRATE)
tedrate = fred.get_series('tedrate', start, end)
tedrate = round(tedrate.resample('MS').mean(),2)
tedrate = pd.DataFrame(tedrate, index = tedrate.index, columns = ['tedrate'])
interest_rates += 1

In [309]:
# 10-Year Treasury Constant Maturity Minus Federal Funds Rate (T10YFF)
t10yff = fred.get_series('T10YFF', start, end)
t10yff = round(t10yff.resample('MS').mean(),2)
t10yff = pd.DataFrame(t10yff, index = t10yff.index, columns = ['t10yff'])
interest_rates += 1

In [310]:
# 5-Year Treasury Constant Maturity Minus Federal Funds Rate (T5YFF)
t5yff = fred.get_series('t5yff', start, end)
t5yff = round(t5yff.resample('MS').mean(),2)
t5yff = pd.DataFrame(t5yff, index = t5yff.index, columns = ['t5yff'])
interest_rates += 1

In [311]:
# 6-Month Treasury Bill Minus Federal Funds Rate (TB6SMFFM)
tb6smffm = fred.get_series('tb6smffm', start, end)
tb6smffm = round(tb6smffm.resample('MS').mean(),2)
tb6smffm = pd.DataFrame(tb6smffm, index = tb6smffm.index, columns = ['tb6smffm'])
interest_rates += 1

In [312]:
# 3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD3MTD156N)
usd3mtd156n = fred.get_series('usd3mtd156n', start, end)
usd3mtd156n = round(usd3mtd156n.resample('MS').mean(),2)
usd3mtd156n = pd.DataFrame(usd3mtd156n, index = usd3mtd156n.index, columns = ['usd3mtd156n'])
interest_rates += 1

In [313]:
# 12-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD12MD156N)
usd12md156n = fred.get_series('usd12md156n', start, end)
usd12md156n = round(usd12md156n.resample('MS').mean(),2)
usd12md156n = pd.DataFrame(usd12md156n, index = usd12md156n.index, columns = ['usd12md156n'])
interest_rates += 1

In [314]:
# 1-Week London Interbank Offered Rate (LIBOR), based on Euro (EUR1WKD156N)
eur1wkd156n = fred.get_series('eur1wkd156n', start, end)
eur1wkd156n = round(eur1wkd156n.resample('MS').mean(),2)
eur1wkd156n = pd.DataFrame(eur1wkd156n, index = eur1wkd156n.index, columns = ['eur1wkd156n'])
interest_rates += 1

In [315]:
# Bank Prime Loan Rate (MPRIME)
mprime = fred.get_series('mprime', start, end)
mprime = round(mprime.resample('MS').mean(),2)
mprime = pd.DataFrame(mprime, index = mprime.index, columns = ['mprime'])
interest_rates += 1

In [316]:
# 3-Month Treasury Bill: Secondary Market Rate (TB3MS)
tb3ms = fred.get_series('tb3ms', start, end)
tb3ms = round(tb3ms.resample('MS').mean(),2)
tb3ms = pd.DataFrame(tb3ms, index = tb3ms.index, columns = ['tb3ms'])
interest_rates += 1

In [317]:
# 1-Year Treasury Bill: Secondary Market Rate (TB1YR)
tb1yr = fred.get_series('tb1yr', start, end)
tb1yr = round(tb1yr.resample('MS').mean(),2)
tb1yr = pd.DataFrame(tb1yr, index = tb1yr.index, columns = ['tb1yr'])
interest_rates += 1

In [318]:
# 6-Month Treasury Bill: Secondary Market Rate (DTB6)
dtb6 = fred.get_series('dtb6', start, end)
dtb6 = round(dtb6.resample('MS').mean(),2)
dtb6= pd.DataFrame(dtb6, index = dtb6.index, columns = ['dtb6'])
interest_rates += 1

In [319]:
# 10-Year Treasury Constant Maturity Rate (DGS10)
dgs10 = fred.get_series('dgs10', start, end)
dgs10 = round(dgs10.resample('MS').mean(),2)
dgs10 = pd.DataFrame(dgs10, index = dgs10.index, columns = ['dgs10'])
interest_rates += 1

In [320]:
# 1-Year Treasury Constant Maturity Rate (DGS1)
dgs1 = fred.get_series('dgs1', start, end)
dgs1 = round(dgs1.resample('MS').mean(),2)
dgs1 = pd.DataFrame(dgs1, index = dgs1.index, columns = ['dgs1'])
interest_rates += 1

In [321]:
# 10-Year Treasury Inflation-Indexed Security, Constant Maturity (DFII10)
dfii10 = fred.get_series('dfii10', start, end)
dfii10 = round(dfii10.resample('MS').mean(),2)
dfii10 = pd.DataFrame(dfii10, index = dfii10.index, columns = ['dfii10'])
interest_rates += 1

In [322]:
# 3-Month Treasury Constant Maturity Rate (DGS3MO)
dgs3mo = fred.get_series('dgs3mo', start, end)
dgs3mo = round(dgs3mo.resample('MS').mean(),2)
dgs3mo = pd.DataFrame(dgs3mo, index = dgs3mo.index, columns = ['dgs3mo'])
interest_rates += 1

In [323]:
# 30-Year Treasury Constant Maturity Rate (DGS30)
dgs30 = fred.get_series('dgs30', start, end)
dgs30 = round(dgs30.resample('MS').mean(),2)
dgs30 = pd.DataFrame(dgs30, index = dgs30.index, columns = ['dgs30'])
interest_rates += 1

In [324]:
exchange_rates = 0

In [325]:
# U.S. / Euro Foreign Exchange Rate (DEXUSEU)
dexuseu = fred.get_series('dexuseu', start, end)
dexuseu = round(dexuseu.resample('MS').mean(),2)
dexuseu = pd.DataFrame(dexuseu, index = dexuseu.index, columns = ['dexuseu'])
exchange_rates += 1

In [326]:
# China / U.S. Foreign Exchange Rate (DEXCHUS)
dexchus = fred.get_series('dexchus', start, end)
dexchus = round(dexchus.resample('MS').mean(),2)
dexchus = pd.DataFrame(dexchus, index = dexchus.index, columns = ['dexchus'])
exchange_rates += 1

In [327]:
# U.S. / U.K. Foreign Exchange Rate (DEXUSUK)
dexusuk = fred.get_series('dexusuk', start, end)
dexusuk = round(dexusuk.resample('MS').mean(),2)
dexusuk = pd.DataFrame(dexusuk, index = dexusuk.index, columns = ['dexusuk'])
exchange_rates += 1

In [328]:
# Singapore / U.S. Foreign Exchange Rate (DEXSIUS)
dexsius = fred.get_series('dexsius', start, end)
dexsius = round(dexsius.resample('MS').mean(),2)
dexsius = pd.DataFrame(dexsius, index = dexsius.index, columns = ['dexsius'])
exchange_rates += 1

In [329]:
financial_indicators = 0

In [330]:
# NASDAQ Composite Index (NASDAQCOM)
nasdaqcom = fred.get_series('nasdaqcom', start, end)
nasdaqcom = round(nasdaqcom.resample('MS').mean(),2)
nasdaqcom = pd.DataFrame(nasdaqcom, index = nasdaqcom.index, columns = ['nasdaqcom'])
financial_indicators += 1

In [331]:
# Wilshire US Real Estate Investment Trust Total Market Index (Wilshire US REIT)(WILLREITIND)
willreitind = fred.get_series('willreitind', start, end)
willreitind = round(willreitind.resample('MS').mean(),2)
willreitind = pd.DataFrame(willreitind, index = willreitind.index, columns = ['willreitind'])
financial_indicators += 1

In [332]:
# Wilshire US Real Estate Securities Price Index (Wilshire US RESI) (WILLRESIPR)
willresipr = fred.get_series('willresipr', start, end)
willresipr = round(willresipr.resample('MS').mean(),2)
willresipr = pd.DataFrame(willresipr, index = willresipr.index, columns = ['willresipr'])
financial_indicators += 1

In [333]:
# Wilshire US Large-Cap Total Market Index (WILLLRGCAP)
willlrgcap = fred.get_series('willlrgcap', start, end)
willlrgcap = round(willlrgcap.resample('MS').mean(),2)
willlrgcap = pd.DataFrame(willlrgcap, index = willlrgcap.index, columns = ['willlrgcap'])
financial_indicators += 1

In [334]:
# Wilshire US Small-Cap Total Market Index (WILLSMLCAP)
willsmlcap = fred.get_series('willsmlcap', start, end)
willsmlcap = round(willsmlcap.resample('MS').mean(),2)
willsmlcap = pd.DataFrame(willsmlcap, index = willsmlcap.index, columns = ['willsmlcap'])
financial_indicators += 1

In [335]:
# CBOE Volatility Index: VIX (VIXCLS)
vixcls = fred.get_series('vixcls', start, end)
vixcls = round(vixcls.resample('MS').mean(),2)
vixcls = pd.DataFrame(vixcls, index = vixcls.index, columns = ['vixcls'])
financial_indicators += 1

In [336]:
# CBOE DJIA Volatility Index (VXDCLS)
vxdcls = fred.get_series('vxdcls', start, end)
vxdcls = round(vxdcls.resample('MS').mean(),2)
vxdcls = pd.DataFrame(vxdcls, index = vxdcls.index, columns = ['vxdcls'])
financial_indicators += 1

In [337]:
# St. Louis Fed Financial Stress Index (STLFSI)
stlfsi = fred.get_series('stlfsi', start, end)
stlfsi = round(stlfsi.resample('MS').mean(),2)
stlfsi = pd.DataFrame(stlfsi, index = stlfsi.index, columns = ['stlfsi'])
financial_indicators += 1

In [338]:
# Chicago Fed National Financial Conditions Index (NFCI)
nfci = fred.get_series('nfci', start, end)
nfci = round(nfci.resample('MS').mean(),2)
nfci = pd.DataFrame(nfci, index = nfci.index, columns = ['nfci'])
financial_indicators += 1

In [339]:
commercial_banking = 0

In [340]:
# Borrowings, All Commercial Banks (H8B3094NCBA)
h8b3094ncba = fred.get_series('h8b3094ncba', start, end)
h8b3094ncba = round(h8b3094ncba.resample('MS').mean(),2)
h8b3094ncba = pd.DataFrame(h8b3094ncba, index = h8b3094ncba.index, columns = ['h8b3094ncba'])
commercial_banking += 1

In [341]:
# Bank Credit, Domestically Chartered Commercial Banks (BC0DCBW027NBOG)
bc0dcbw027nbog = fred.get_series('bc0dcbw027nbog', start, end)
bc0dcbw027nbog = round(bc0dcbw027nbog.resample('MS').mean(),2)
bc0dcbw027nbog = pd.DataFrame(bc0dcbw027nbog, index = bc0dcbw027nbog.index, columns = ['bc0dcbw027nbog'])
commercial_banking += 1

In [342]:
# Cash Assets, Domestically Chartered Commercial Banks (CASDCBW027NBOG)
casdcbw027nbog = fred.get_series('casdcbw027nbog', start, end)
casdcbw027nbog = round(casdcbw027nbog.resample('MS').mean(),2)
casdcbw027nbog = pd.DataFrame(casdcbw027nbog, index = casdcbw027nbog.index, columns = ['casdcbw027nbog'])
commercial_banking += 1

In [343]:
# Commercial and Industrial Loans, Domestically Chartered Commercial Banks (CILDCBW027NBOG)
commercial_banking += 1
cildcbw027nbog = fred.get_series('cildcbw027nbog', start, end)
cildcbw027nbog = round(cildcbw027nbog.resample('MS').mean(),2)
cildcbw027nbog = pd.DataFrame(cildcbw027nbog, index = cildcbw027nbog.index, columns = ['cildcbw027nbog'])

In [344]:
# Deposits, Domestically Chartered Commercial Banks (DPSDCBW027NBOG)
commercial_banking += 1
dpsdcbw027nbog = fred.get_series('dpsdcbw027nbog', start, end)
dpsdcbw027nbog = round(dpsdcbw027nbog.resample('MS').mean(),2)
dpsdcbw027nbog = pd.DataFrame(dpsdcbw027nbog, index = dpsdcbw027nbog.index, columns = ['dpsdcbw027nbog'])

In [345]:
# Real Estate Loans: Residential Real Estate Loans: Revolving Home Equity Loans, Domestically Chartered Commercial Banks (RHEDCBW027NBOG)
commercial_banking += 1
rhedcbw027nbog = fred.get_series('rhedcbw027nbog', start, end)
rhedcbw027nbog = round(rhedcbw027nbog.resample('MS').mean(),2)
rhedcbw027nbog = pd.DataFrame(rhedcbw027nbog, index = rhedcbw027nbog.index, columns = ['rhedcbw027nbog'])

In [346]:
# Consumer Loans, All Commercial Banks (CLSACBW027NBOG)
commercial_banking += 1
clsacbw027nbog = fred.get_series('clsacbw027nbog', start, end)
clsacbw027nbog = round(clsacbw027nbog.resample('MS').mean(),2)
clsacbw027nbog = pd.DataFrame(clsacbw027nbog, index = clsacbw027nbog.index, columns = ['clsacbw027nbog'])

In [347]:
# Deposits, All Commercial Banks (DPSACBW027NBOG)
commercial_banking += 1
dpsacbw027nbog = fred.get_series('dpsacbw027nbog', start, end)
dpsacbw027nbog = round(dpsacbw027nbog.resample('MS').mean(),2)
dpsacbw027nbog = pd.DataFrame(dpsacbw027nbog, index = dpsacbw027nbog.index, columns = ['dpsacbw027nbog'])

In [348]:
# Real Estate Loans, All Commercial Banks (RELACBW027NBOG)
commercial_banking += 1
relacbw027nbog = fred.get_series('relacbw027nbog', start, end)
relacbw027nbog = round(relacbw027nbog.resample('MS').mean(),2)
relacbw027nbog = pd.DataFrame(relacbw027nbog, index = relacbw027nbog.index, columns = ['relacbw027nbog'])

In [349]:
# Real Estate Loans: Residential Real Estate Loans: Revolving Home Equity Loans, All Commercial Banks (RHEACBW027NBOG)
commercial_banking += 1
rheacbw027nbog = fred.get_series('rheacbw027nbog', start, end)
rheacbw027nbog = round(rheacbw027nbog.resample('MS').mean(),2)
rheacbw027nbog = pd.DataFrame(rheacbw027nbog, index = rheacbw027nbog.index, columns = ['rheacbw027nbog'])

In [350]:
# Total Assets, All Commercial Banks (TLAACBW027NBOG)
commercial_banking += 1
tlaacbw027nbog = fred.get_series('tlaacbw027nbog', start, end)
tlaacbw027nbog = round(tlaacbw027nbog.resample('MS').mean(),2)
tlaacbw027nbog = pd.DataFrame(tlaacbw027nbog, index = tlaacbw027nbog.index, columns = ['tlaacbw027nbog'])

In [351]:
# Total Liabilities, All Commercial Banks (TLBACBW027NBOG)
commercial_banking += 1
tlbacbw027nbog = fred.get_series('tlbacbw027nbog', start, end)
tlbacbw027nbog = round(tlbacbw027nbog.resample('MS').mean(),2)
tlbacbw027nbog = pd.DataFrame(tlbacbw027nbog, index = tlbacbw027nbog.index, columns = ['tlbacbw027nbog'])

In [352]:
# Commercial and Industrial Loans, All Commercial Banks (TOTCI)
commercial_banking += 1
totci = fred.get_series('totci', start, end)
totci = round(totci.resample('MS').mean(),2)
totci = pd.DataFrame(totci, index = totci.index, columns = ['totci'])

In [353]:
academic_data = 0

In [354]:
# Economic Policy Uncertainty Index for United States (USEPUINDXD)
academic_data += 1
usepuindxd = fred.get_series('usepuindxd', start, end)
usepuindxd = round(usepuindxd.resample('MS').mean(),2)
usepuindxd = pd.DataFrame(usepuindxd, index = usepuindxd.index, columns = ['usepuindxd'])

In [355]:
# Equity Market-related Economic Uncertainty Index (WLEMUINDXD)
academic_data += 1
wlemuindxd = fred.get_series('wlemuindxd', start, end)
wlemuindxd = round(wlemuindxd.resample('MS').mean(),2)
wlemuindxd = pd.DataFrame(wlemuindxd, index = wlemuindxd.index, columns = ['wlemuindxd'])

In [356]:
# Equity Market Volatility Tracker: Financial Crises (EMVFINCRISES)
academic_data += 1
emvfincrises = fred.get_series('emvfincrises', start, end)
emvfincrises = round(emvfincrises.resample('MS').mean(),2)
emvfincrises = pd.DataFrame(emvfincrises, index = emvfincrises.index, columns = ['emvfincrises'])

In [357]:
# Price Pressures Measure (STLPPM)
prices = 1
stlppm = fred.get_series('stlppm', start, end)
stlppm = round(stlppm.resample('MS').mean(),2)
stlppm = pd.DataFrame(stlppm, index = stlppm.index, columns = ['stlppm'])

In [358]:
rates_list = [usd1mtd156n, mortgage30us, mortgage15us, dcpf3m, cpn3m, dcpf1m, aaa, baa, dff, flexsc, t10y2y, t10y3m, tedrate, t10yff, t5yff, tb6smffm, usd3mtd156n, usd12md156n, 
              eur1wkd156n, mprime, tb3ms, tb1yr, dtb6, dgs10, dfii10, dgs3mo, dgs30, dexuseu, dexchus, dexusuk, dexsius, nasdaqcom, willreitind, willlrgcap, willsmlcap, vxdcls, 
              stlfsi, nfci, h8b3094ncba, bc0dcbw027nbog, casdcbw027nbog, cildcbw027nbog, dpsdcbw027nbog, rhedcbw027nbog, clsacbw027nbog, dpsacbw027nbog, relacbw027nbog, rheacbw027nbog, 
              tlaacbw027nbog, tlbacbw027nbog, totci, usepuindxd, wlemuindxd, emvfincrises, stlppm]
rates_df = pd.concat(rates_list, axis = 1)
rates_df

,usd1mtd156n,mortgage30us,mortgage15us,dcpf3m,cpn3m,dcpf1m,aaa,baa,dff,flexsc,...,dpsacbw027nbog,relacbw027nbog,rheacbw027nbog,tlaacbw027nbog,tlbacbw027nbog,totci,usepuindxd,wlemuindxd,emvfincrises,stlppm
2000-03-01,6.05,8.24,7.83,6.03,6.00,5.94,7.68,8.37,5.85,5.93,...,3521.57,1510.16,107.31,5685.93,5198.06,1024.58,64.99,46.42,0.90,0.19
2000-04-01,6.15,8.15,7.80,6.15,6.11,6.03,7.64,8.40,6.02,6.15,...,3584.93,1531.49,110.75,5747.53,5262.45,1033.44,59.76,88.46,1.04,0.65
2000-05-01,6.54,8.51,8.18,6.57,6.54,6.41,7.99,8.90,6.27,6.30,...,3563.07,1559.57,113.35,5821.97,5336.37,1051.14,77.52,55.96,0.63,0.30
2000-06-01,6.65,8.29,7.99,6.59,6.57,6.53,7.67,8.48,6.53,6.62,...,3581.19,1578.90,114.16,5841.58,5345.38,1062.59,67.24,74.91,0.52,0.46
2000-07-01,6.63,8.15,7.86,6.53,6.52,6.50,7.65,8.35,6.54,6.62,...,3607.03,1596.80,118.27,5861.82,5360.64,1068.18,53.87,45.33,0.55,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-01,2.04,3.61,3.12,2.03,1.95,2.02,3.03,3.91,2.04,2.08,...,12921.51,4542.70,329.93,17489.44,15505.06,2364.67,118.31,66.56,2.05,0.03
2019-10-01,1.88,3.69,3.14,1.88,1.80,1.83,3.01,3.92,1.83,1.90,...,13048.47,4566.61,326.66,17577.01,15607.18,2356.41,98.00,70.15,2.23,0.03
2019-11-01,1.74,3.70,3.16,1.77,1.62,1.61,3.06,3.94,1.55,1.72,...,13200.03,4587.38,324.00,17727.80,15755.83,2363.16,81.04,54.55,1.65,0.05
2019-12-01,1.75,3.72,3.18,1.76,1.70,1.65,3.01,3.88,1.55,1.70,...,13329.48,4617.34,321.94,17860.64,15875.90,2359.06,121.31,82.88,3.78,0.05


## Clean Rates DF

In [359]:
rates_df.loc[:, rates_df.isna().any()]

,cpn3m,tb1yr,dfii10,dgs30
2000-03-01,6.00,5.86,NaN,6.05
2000-04-01,6.11,5.80,NaN,5.85
2000-05-01,6.54,5.94,NaN,6.15
2000-06-01,6.57,5.83,NaN,5.93
2000-07-01,6.52,5.75,NaN,5.85
...,...,...,...,...
2019-09-01,1.95,1.75,0.11,2.16
2019-10-01,1.80,1.57,0.15,2.19
2019-11-01,1.62,1.53,0.17,2.28
2019-12-01,1.70,1.51,0.14,2.30


In [360]:
rates_df.drop(labels = ['tb1yr', 'dgs30', 'dfii10'], axis = 1, inplace = True)

In [361]:
rates_df.cpn3m = rates_df.cpn3m.ffill()

In [362]:
AMT_rates = pd.concat([AMT, rates_df], axis = 1)
ELS_rates = pd.concat([ELS, rates_df], axis = 1)
PLD_rates = pd.concat([PLD, rates_df], axis = 1)
FR_rates = pd.concat([FR, rates_df], axis = 1)
MAA_rates = pd.concat([MAA, rates_df], axis = 1)
SUI_rates = pd.concat([SUI, rates_df], axis = 1)
BXMT_rates = pd.concat([BXMT, rates_df], axis = 1)
RHP_rates = pd.concat([RHP, rates_df], axis = 1)

In [363]:
AMT_rates.to_csv('data/AMT_rates.csv', index = True, index_label = 'date')
ELS_rates.to_csv('data/ELS_rates.csv', index = True, index_label = 'date')
ELS_rates.to_csv('data/ELS_rates.csv', index = True, index_label = 'date')
PLD_rates.to_csv('data/PLD_rates.csv', index = True, index_label = 'date')
FR_rates.to_csv('data/FR_rates.csv', index = True, index_label = 'date')
MAA_rates.to_csv('data/MAA_rates.csv', index = True, index_label = 'date')
SUI_rates.to_csv('data/SUI_rates.csv', index = True, index_label = 'date')
BXMT_rates.to_csv('data/BXMT_rates.csv', index = True, index_label = 'date')
RHP_rates.to_csv('data/RHP_rates.csv', index = True, index_label = 'date')

## Perform Lasso Regression

In [364]:
def rmse_cv(model, X, y):
    rmse= np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [365]:
def lasso_reg(dataframe):
    
    target = dataframe['mid']
    dataframe = dataframe.drop(['mid'], axis = 1)
    
    scale = MinMaxScaler()
    rates_trans = scale.fit_transform(dataframe)
    
    model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(dataframe, target)
    rmse_cv(model_lasso, dataframe, target).mean()
    
    print(f"The R-score of the Lasso Regression is: {model_lasso.score(dataframe, target)}")
    print('')
    
    coef = pd.Series(model_lasso.coef_, index = dataframe.columns)
    
    print(coef)
    print(' ')
    print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " + str(sum(coef == 0)) + " variables")
    
    imp_coef = pd.concat([coef.sort_values().head(26),
                          coef.sort_values().tail(26)])
    matplotlib.rcParams['figure.figsize'] = (8,8)
    imp_coef.plot(kind = 'barh')
    plt.title("Coefficients in the Lasso Model")
    plt.show()
    
    matplotlib.rcParams['figure.figsize'] = (8,8)
    preds = pd.DataFrame({"preds":model_lasso.predict(dataframe), "true":target})
    preds["residuals"] = preds["true"] - preds["preds"]
    preds.plot(x = "preds", y = "residuals",kind = "scatter")

## Feature Correlation

In [366]:
AMT_rates.corr()

,mid,usd1mtd156n,mortgage30us,mortgage15us,dcpf3m,cpn3m,dcpf1m,aaa,baa,dff,...,dpsacbw027nbog,relacbw027nbog,rheacbw027nbog,tlaacbw027nbog,tlbacbw027nbog,totci,usepuindxd,wlemuindxd,emvfincrises,stlppm
mid,1.000000,-0.225955,-0.666461,-0.641091,-0.211450,-0.209843,-0.220564,-0.760446,-0.768149,-0.214585,...,0.912405,0.762015,0.098065,0.880730,0.881235,0.935653,-0.074704,-0.262973,0.249534,-0.245551
usd1mtd156n,-0.225955,1.000000,0.779643,0.810060,0.998031,0.996078,0.999030,0.616571,0.504284,0.992636,...,-0.500571,-0.428609,-0.467894,-0.510065,-0.507469,-0.300220,-0.314584,0.127540,-0.418598,0.497235
mortgage30us,-0.666461,0.779643,1.000000,0.995705,0.771963,0.769957,0.777942,0.953482,0.875461,0.767834,...,-0.860235,-0.782821,-0.549704,-0.863011,-0.859918,-0.689288,-0.169587,0.306035,-0.451021,0.373266
mortgage15us,-0.641091,0.810060,0.995705,1.000000,0.804043,0.800191,0.807917,0.936545,0.871190,0.795707,...,-0.842914,-0.745750,-0.527793,-0.842632,-0.839467,-0.662495,-0.163135,0.299965,-0.444252,0.381897
dcpf3m,-0.211450,0.998031,0.771963,0.804043,1.000000,0.997034,0.998048,0.601282,0.491780,0.991422,...,-0.483515,-0.407860,-0.456823,-0.492526,-0.489798,-0.282148,-0.329664,0.105044,-0.424467,0.507692
cpn3m,-0.209843,0.996078,0.769957,0.800191,0.997034,1.000000,0.998014,0.601874,0.481063,0.995881,...,-0.488601,-0.427979,-0.479977,-0.502100,-0.499613,-0.294440,-0.344015,0.101498,-0.454105,0.494322
dcpf1m,-0.220564,0.999030,0.777942,0.807917,0.998048,0.998014,1.000000,0.614186,0.496617,0.996327,...,-0.498662,-0.432152,-0.477438,-0.510112,-0.507594,-0.299383,-0.327044,0.118679,-0.439880,0.492701
aaa,-0.760446,0.616571,0.953482,0.936545,0.601282,0.601874,0.614186,1.000000,0.943275,0.606372,...,-0.903094,-0.848193,-0.521657,-0.902271,-0.899662,-0.758389,-0.021718,0.402271,-0.362431,0.263974
baa,-0.768149,0.504284,0.875461,0.871190,0.491780,0.481063,0.496617,0.943275,1.000000,0.475276,...,-0.857044,-0.742934,-0.376462,-0.834410,-0.830774,-0.701418,0.129526,0.451594,-0.203057,0.193821
dff,-0.214585,0.992636,0.767834,0.795707,0.991422,0.995881,0.996327,0.606372,0.475276,1.000000,...,-0.498081,-0.446262,-0.497560,-0.513729,-0.511544,-0.304989,-0.344761,0.106826,-0.485543,0.477214


In [367]:
print(f"Interest Rates = {interest_rates}")
print(f"Exchange Rates = {exchange_rates}")
print(f"Financial Indicators = {financial_indicators}")
print(f"Commercial Banking = {commercial_banking}")
print(f"Academic Data = {academic_data}")

print(f"Prices = {prices}")

Interest Rates = 28
Exchange Rates = 4
Financial Indicators = 9
Commercial Banking = 13
Academic Data = 3
Prices = 1


In [368]:
interest_rates + exchange_rates + commercial_banking + academic_data + financial_indicators + prices

58

In [369]:
FR_rates

,mid,usd1mtd156n,mortgage30us,mortgage15us,dcpf3m,cpn3m,dcpf1m,aaa,baa,dff,...,dpsacbw027nbog,relacbw027nbog,rheacbw027nbog,tlaacbw027nbog,tlbacbw027nbog,totci,usepuindxd,wlemuindxd,emvfincrises,stlppm
2000-03-01,26.998250,6.05,8.24,7.83,6.03,6.00,5.94,7.68,8.37,5.85,...,3521.57,1510.16,107.31,5685.93,5198.06,1024.58,64.99,46.42,0.90,0.19
2000-04-01,27.723158,6.15,8.15,7.80,6.15,6.11,6.03,7.64,8.40,6.02,...,3584.93,1531.49,110.75,5747.53,5262.45,1033.44,59.76,88.46,1.04,0.65
2000-05-01,29.697273,6.54,8.51,8.18,6.57,6.54,6.41,7.99,8.90,6.27,...,3563.07,1559.57,113.35,5821.97,5336.37,1051.14,77.52,55.96,0.63,0.30
2000-06-01,30.174318,6.65,8.29,7.99,6.59,6.57,6.53,7.67,8.48,6.53,...,3581.19,1578.90,114.16,5841.58,5345.38,1062.59,67.24,74.91,0.52,0.46
2000-07-01,30.854000,6.63,8.15,7.86,6.53,6.52,6.50,7.65,8.35,6.54,...,3607.03,1596.80,118.27,5861.82,5360.64,1068.18,53.87,45.33,0.55,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-01,39.227875,2.04,3.61,3.12,2.03,1.95,2.02,3.03,3.91,2.04,...,12921.51,4542.70,329.93,17489.44,15505.06,2364.67,118.31,66.56,2.05,0.03
2019-10-01,40.290109,1.88,3.69,3.14,1.88,1.80,1.83,3.01,3.92,1.83,...,13048.47,4566.61,326.66,17577.01,15607.18,2356.41,98.00,70.15,2.23,0.03
2019-11-01,41.975713,1.74,3.70,3.16,1.77,1.62,1.61,3.06,3.94,1.55,...,13200.03,4587.38,324.00,17727.80,15755.83,2363.16,81.04,54.55,1.65,0.05
2019-12-01,41.489007,1.75,3.72,3.18,1.76,1.70,1.65,3.01,3.88,1.55,...,13329.48,4617.34,321.94,17860.64,15875.90,2359.06,121.31,82.88,3.78,0.05
